## 평가자-최적화기 워크플로우
이 워크플로우에서는 하나의 LLM 호출이 응답을 생성하고, 다른 하나가 루프에서 평가와 피드백을 제공합니다.

### 이 워크플로우를 사용해야 하는 경우
이 워크플로우는 다음과 같은 경우에 특히 효과적입니다:

- 명확한 평가 기준이 있을 때
- 반복적인 개선에서 가치를 얻을 수 있을 때

적합한 두 가지 신호는 다음과 같습니다:

- 피드백이 제공되면 LLM 응답이 명백하게 개선될 수 있음
- LLM이 의미 있는 피드백을 스스로 제공할 수 있음

In [ ]:
from util import llm_call, extract_xml


def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """피드백을 기반으로 솔루션을 생성하고 개선합니다."""
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"
    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")

    print("\n=== 생성 시작 ===")
    print(f"생각:\n{thoughts}\n")
    print(f"생성됨:\n{result}")
    print("=== 생성 종료 ===\n")

    return thoughts, result


def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """솔루션이 요구사항을 충족하는지 평가합니다."""
    full_prompt = f"{prompt}\n원래 작업: {task}\n평가할 내용: {content}"
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")

    print("=== 평가 시작 ===")
    print(f"상태: {evaluation}")
    print(f"피드백: {feedback}")
    print("=== 평가 종료 ===\n")

    return evaluation, feedback


def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """요구사항이 충족될 때까지 생성과 평가를 반복합니다."""
    memory = []
    chain_of_thought = []

    thoughts, result = generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})

    while True:
        evaluation, feedback = evaluate(evaluator_prompt, result, task)
        if evaluation == "PASS":
            return result, chain_of_thought

        context = "\n".join(
            ["이전 시도:", *[f"- {m}" for m in memory], f"\n피드백: {feedback}"]
        )

        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

### 예제 사용 사례: 반복적인 코딩 루프



In [ ]:
evaluator_prompt = """
다음 코드 구현을 평가하세요:
1. 코드 정확성
2. 시간 복잡도
3. 스타일 및 모범 사례

평가만 수행하고 작업을 해결하려고 시도하지 마세요.
모든 기준이 충족되고 더 이상 개선 제안이 없을 때만 "PASS"를 출력하세요.
다음 형식으로 평가를 간결하게 출력하세요.

<evaluation>PASS, NEEDS_IMPROVEMENT, 또는 FAIL</evaluation>
<feedback>
무엇이 개선되어야 하는지와 그 이유.
</feedback>
"""

generator_prompt = """
당신의 목표는 <사용자 입력>을 기반으로 작업을 완료하는 것입니다. 이전 생성에서 피드백이 있다면,
솔루션을 개선하기 위해 이를 반영해야 합니다.

다음 형식으로 답변을 간결하게 출력하세요: 

<thoughts>
[작업과 피드백에 대한 이해, 그리고 개선 계획]
</thoughts>

<response>
[여기에 코드 구현]
</response>
"""

task = """
<사용자 입력>
다음 기능을 가진 Stack을 구현하세요:
1. push(x)
2. pop()
3. getMin()
모든 연산은 O(1)이어야 합니다.
</사용자 입력>
"""

loop(task, evaluator_prompt, generator_prompt)